In [1]:
import json
import sys
import os
import random
import gradio as gr
sys.path.insert(0, '../../')
from data_loader import get_data_to_load, split_json_and_image_files, load_json_files, load_image_file_raw
from countryconverter import convert_country_to_correct_name

list_files = get_data_to_load(loading_file='../04_data_cleaning/updated_data_list', file_location='../01_enriching/.data/', image_file_location='../../1_data_collection/.data/', allow_new_file_creation=False, from_remote_only=True, download_link='default', limit=100, shuffle_seed=33, allow_file_location_env=True, allow_image_file_location_env=True, allow_json_file_location_env=True, allow_download_link_env=True)

json_files, image_files = split_json_and_image_files(list_files)

labels = load_json_files(json_files)
countries = [item['country_name'] for item in labels]
coordinates = [item['coordinates'] for item in labels]

countries_to_image = list(zip(countries, image_files))

/Users/linus/gitprojects/dspro2/.venv/lib/python3.10/site-packages/fuzzywuzzy/fuzz.py:11: UserWarning: Using slow pure-python SequenceMatcher. Install python-Levenshtein to remove this warning
  warnings.warn('Using slow pure-python SequenceMatcher. Install python-Levenshtein to remove this warning')


All local files: 421768
Relevant files: 421768
Limited files: 200


In [2]:
# Seed for randomization
seed = 42
random.seed(seed)
random.shuffle(countries_to_image)

# Global index to keep track of the current image
current_index = 0
# If empty string guesses were already made and we need to load the next image
solution = ""

num_guesses = 0
top_1_correct = 0
top_3_correct = 0
top_5_correct = 0

def get_country_name(country):
    if not country:
        return ""
    try:
        return convert_country_to_correct_name(country).name
    except:
        return ""

# Function to handle the input and generate output
def process_guesses(image, *inputs):
    global solution, top_1_correct, top_3_correct, top_5_correct, num_guesses
    # filter out empty inputs
    actual_inputs = [input for input in inputs if input]
    num_inputs = len(actual_inputs)
    num_processed_inputs = 0
    
    if num_inputs:
      # process the inputs to countries
      inputs = [get_country_name(input) for input in inputs]
      actual_processed_inputs = [input for input in inputs if input]
      num_processed_inputs = len(actual_processed_inputs)
      
    all_found = False
    
    if num_processed_inputs == num_inputs:
      all_found = True
      current_solution = solution
      solution = ""
    
      num_guesses += 1
    
      if inputs[0] == current_solution:
          top_1_correct += 1
          
      if current_solution in inputs[:3]:
          top_3_correct += 1
          
      if current_solution in inputs:
          top_5_correct += 1
          
      results = f"Top 1: {top_1_correct/num_guesses}, Top 3: {top_3_correct/num_guesses}, Top 5: {top_5_correct/num_guesses}, Total guesses: {num_guesses}"
      
      # combine with current_solution
      output = f"Correct answer: {current_solution}\n{results}"
    
    return image, output if all_found else "Invalid guess", gr.Textbox(label="Input 1", interactive=not all_found), gr.Textbox(label="Input 2", interactive=not all_found), gr.Textbox(label="Input 3", interactive=not all_found), gr.Textbox(label="Input 4", interactive=not all_found), gr.Textbox(label="Input 5", interactive=not all_found), gr.Button("Next") if all_found else gr.Button("Guess")

# Function to load the next image
def load_next_image():
    global current_index, solution
    current_index = (current_index + 1) % len(countries_to_image)
    next_country, next_image_path = countries_to_image[current_index]
    next_image = load_image_file_raw(next_image_path)
    solution = next_country
    return next_image, f"Top 1: {top_1_correct/num_guesses}, Top 3: {top_3_correct/num_guesses}, Top 5: {top_5_correct/num_guesses}, Total guesses: {num_guesses}" if num_guesses else "", gr.Textbox(label="Input 1", interactive=True, value=""), gr.Textbox(label="Input 2", interactive=True, value=""), gr.Textbox(label="Input 3", interactive=True, value=""), gr.Textbox(label="Input 4", interactive=True, value=""), gr.Textbox(label="Input 5", interactive=True, value=""), gr.Button("Guess")

# Function to load the next image
def progress_ui_state(image, *inputs):
    global current_index, solution
    if solution:
      return process_guesses(image, *inputs)
    else:
      return load_next_image()

with gr.Blocks() as demo:
    with gr.Row():
        with gr.Column():
            image_output = gr.Image(type="pil", label="Image", interactive=False)
            result_output = gr.Textbox(label="Result", interactive=False)
        with gr.Column():
            text_inputs = [
                gr.Textbox(label="Input 1"),
                gr.Textbox(label="Input 2"),
                gr.Textbox(label="Input 3"),
                gr.Textbox(label="Input 4"),
                gr.Textbox(label="Input 5")
            ]
            submit_button = gr.Button("Next")

    # Initial image load
    demo.load(progress_ui_state, inputs=[image_output] + text_inputs, outputs=[image_output, result_output] + text_inputs + [submit_button])

    # Guessing or loading next image
    submit_button.click(progress_ui_state, inputs=[image_output] + text_inputs, outputs=[image_output, result_output] + text_inputs + [submit_button])

# Launch the Gradio app
demo.launch()

Running on local URL:  http://127.0.0.1:7860

To create a public link, set `share=True` in `launch()`.


SubdivisionHierarchy(code='US-GA', country_code='US', name='Georgia', parent_code=None, type='State')
SubdivisionHierarchy(code='MX-MEX', country_code='MX', name='México', parent_code=None, type='State')


### Linus Results

Top 1:0.26666666666666666, Top 3:0.43333333333333335, Top 5:0.5666666666666667, Total guesses: 30